### Mistral-7B Finetuning 실습
- 한국어 기사 데이터셋으로 미세 조정하여 한국어 기사 본문을 요약하는 모델 생성하고 rough 지표로 평가하기
- 데이터셋 출처: https://huggingface.co/datasets/daekeun-ml/naver-news-summarization-ko

### 1. 데이터 준비

In [ ]:
# 데이터셋 로드 및 샘플링
!pip install datasets -qqq
from datasets import load_dataset
train_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='train').shuffle().select(range(2000))
eval_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='validation').shuffle().select(range(200))
test_dataset = load_dataset("daekeun-ml/naver-news-summarization-ko", split='test').shuffle().select(range(200))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/8.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

In [ ]:
# 불필요한 컬럼 제거
train_dataset = train_dataset.remove_columns(['date', 'category', 'press', 'title', 'link'])
eval_dataset = eval_dataset.remove_columns(['date', 'category', 'press', 'title', 'link'])
test_dataset = test_dataset.remove_columns(['date', 'category', 'press', 'title', 'link'])

In [ ]:
print(train_dataset, eval_dataset, test_dataset)

Dataset({
    features: ['document', 'summary'],
    num_rows: 2000
}) Dataset({
    features: ['document', 'summary'],
    num_rows: 200
}) Dataset({
    features: ['document', 'summary'],
    num_rows: 200
})


### QLoRA 설정 / 모델 및 토크나이저 준비

In [ ]:
# 허깅페이스 로그인
from huggingface_hub import login

# 허깅페이스 허브 로그인
token = "****"  # 허깅페이스 액세스 토큰 입력
login(token=token)

In [ ]:
!pip install bitsandbytes -qqq
import torch
import bitsandbytes
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    Trainer,
    TrainingArguments,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

# 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# LoRA 설정
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],
    inference_mode=False,
    init_lora_weights="gaussian"
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.2 MB/s eta 0:00:00


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
# 모델 불러와서 QLoRA 적용하기
model_id = "mistralai/Mistral-7B-v0.1"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True,
    torch_dtype=torch.float16  # float16으로 설정하여 메모리 효율성 향상
)

# 모델 준비
model.config.use_cache = False  # gradient checkpointing을 위해 cache 비활성화
model.enable_input_require_grads()  # 입력에 대한 그래디언트 활성화

# LoRA 적용
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# 토크나이저 설정
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # eos 토큰을 패딩 토큰으로 설정
tokenizer.padding_side = 'right'           # 오른쪽 패딩

tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

### 데이터셋 대화형 포맷으로 변환 및 토큰화 (전처리)

In [ ]:
# instruction 템플릿 정의
def create_prompt_template(document):
    return f"""아래 뉴스 기사를 요약해주세요:

기사 내용:
{document}

요약: """

def create_completion_template(summary):
    return f"{summary}\n"

# 데이터 전처리 함수
def preprocess_function(examples):
    # instruction과 completion 생성
    prompts = [create_prompt_template(doc) for doc in examples["document"]]
    completions = [create_completion_template(summary) for summary in examples["summary"]]

    # 전체 텍스트 생성 (instruction + completion)
    texts = [f"[INST]{prompt}[/INST]{completion}" for prompt, completion in zip(prompts, completions)]

    # 토큰화
    tokenized = tokenizer(texts, truncation=True)

    # labels 설정 (input_ids와 동일)
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized

# 데이터셋 전처리 적용
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
eval_dataset = eval_dataset.map(preprocess_function, batched=True, remove_columns=eval_dataset.column_names)
test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset, eval_dataset, test_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2000
}) Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 200
}) Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 200
})


### 학습 어규먼트 설정 및 모델 학습하기

In [ ]:
# wandb 설정
import wandb
wandb.login(key='77a08abc9cfd76e4b78603826bd7a863487240ac')
run = wandb.init(project='Korean News Summarization', job_type='training', anonymous='allow')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dgriii0606 (dg-test) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
# 트레이너 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,

    # 배치 크기 및 그래디언트 누적
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,

    # 학습률 및 스케줄링
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,

    # 옵티마이저 설정
    optim="adamw_torch",
    weight_decay=0.01,

    # 평가 및 저장 전략
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,

    # 학습 최적화
    fp16=True,
    gradient_checkpointing=True,

    # 로깅 설정
    logging_dir='./logs',
    logging_steps=10,
    report_to=["wandb"],

    # 기타 최적화
    dataloader_num_workers=2,     # 워커 수 감소
    group_by_length=True,
    remove_unused_columns=True,
    load_best_model_at_end=True,

    # 추가 설정
    ddp_find_unused_parameters=False,  # DDP 최적화
    torch_compile=False,               # 컴파일 비활성화로 안정성 향상
)

In [ ]:
# 콜레이터 설정
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)

In [ ]:
# 트레이너 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# 학습 실행
trainer.train()

Step,Training Loss,Validation Loss
50,1.154900,1.099299
100,1.111600,1.070850


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=125, training_loss=1.117371265411377, metrics={'train_runtime': 1422.6097, 'train_samples_per_second': 1.406, 'train_steps_per_second': 0.088, 'total_flos': 1.1682748027694285e+17, 'train_loss': 1.117371265411377, 'epoch': 1.0})

### 모델 평가하기

In [ ]:
# 테스트 데이터셋으로 모델 평가
!pip install rouge_score -qqq
import numpy as np
from rouge_score import rouge_scorer
from tqdm.auto import tqdm

def evaluate_model_metrics(model, tokenizer, test_dataset):
    """모델 성능 평가"""

    # ROUGE 스코어 초기화
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # 평가 결과 저장용 리스트
    rouge_scores = {
        'rouge1': [],
        'rouge2': [],
        'rougeL': []
    }

    # 평가 진행
    model.eval()
    for i in tqdm(range(len(test_dataset))):
        # 인코딩되었던 eval_dataset의 'input_ids', 'label'을 원본 텍스트로 디코딩
        document = tokenizer.decode(test_dataset[i]['input_ids'], skip_special_tokens=True)
        reference = tokenizer.decode(test_dataset[i]['labels'],  skip_special_tokens=True)

        # 모델 예측
        try:
            prompt = f"[INST]아래 뉴스 기사를 요약해주세요:\n기사 내용:\n{document}\n요약:[/INST]"
            inputs = tokenizer([prompt], return_tensors="pt").to("cuda:0")

            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=128,
                    do_sample=True,
                    temperature=0.7,
                    pad_token_id=tokenizer.eos_token_id
                )

            predicted = tokenizer.decode(outputs[0], skip_special_tokens=True)
            predicted = predicted.split("[/INST]")[-1].strip()

            # ROUGE 점수 계산
            scores = scorer.score(reference, predicted)
            rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
            rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
            rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

        except Exception as e:
            print(f"Error processing sample {i}: {str(e)}")
            continue

    # 평균 점수 계산
    avg_scores = {
        'rouge1': np.mean(rouge_scores['rouge1']),
        'rouge2': np.mean(rouge_scores['rouge2']),
        'rougeL': np.mean(rouge_scores['rougeL'])
    }

    # 결과 출력
    print("\n=== 모델 평가 결과 ===")
    print(f"평가한 샘플 수: {len(rouge_scores['rouge1'])}")
    print(f"ROUGE-1: {avg_scores['rouge1']:.4f}")
    print(f"ROUGE-2: {avg_scores['rouge2']:.4f}")
    print(f"ROUGE-L: {avg_scores['rougeL']:.4f}")

    return avg_scores

# 사용 예시
scores = evaluate_model_metrics(model, tokenizer, test_dataset)

  0%|          | 0/200 [00:00<?, ?it/s]


=== 모델 평가 결과 ===
평가한 샘플 수: 200
ROUGE-1: 0.2403
ROUGE-2: 0.1689
ROUGE-L: 0.2359


### 모델 저장 및 허깅페이스 허브 업로드

In [ ]:
# 모델 저장
trainer.model.save_pretrained("mistral-7b-ko-news-summarizer")

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델 정보
base_model_id = "mistralai/Mistral-7B-v0.1"  # 기본 모델 ID
lora_model_id = "mistral-7b-ko-news-summarizer"  # Hugging Face에 업로드할 모델 이름
device_map = "auto"  # 자동으로 GPU/CPU 할당

# 기본 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    low_cpu_mem_usage=True,       # CPU 메모리 사용 절약
    return_dict=True,             # 모델 출력을 dictionary 형식으로 반환
    torch_dtype=torch.float16,     # FP16(16비트 부동소수점) 사용 -> 메모리 절약
    device_map=device_map,         # 자동으로 CPU/GPU 분배
    offload_folder="offload",      # 일부 가중치를 CPU로 오프로드하여 메모리 절약
)

# LoRA 어댑터 로드 및 병합
model = PeftModel.from_pretrained(base_model, lora_model_id)
model = model.merge_and_unload()  # LoRA 어댑터를 기본 모델에 병합 후 언로드 (메모리 절약)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Hugging Face Hub에 모델 및 토크나이저 업로드
model.push_to_hub(lora_model_id)
tokenizer.push_to_hub(lora_model_id)

### 업로드한 모델 불러와서 사용해보기

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

my_model_id = "edgeun/mistral-7b-ko-news-summarizer"

# 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(my_model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(my_model_id)

# 토크나이저 패딩 토큰 설정
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
from transformers import TextStreamer
# 추론 함수 생성
def stream_summary(article):
    prompt = f"[INST]아래 뉴스 기사를 요약해주세요:\n기사 내용:\n{article}\n요약:[/INST]"
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda:0")
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=128, do_sample=True, temperature=0.7)

In [12]:
# 기사 텍스트 입력 및 함수 사용
new_article = """

정월대보름(2월12일)을 앞두고 서민들의 시름이 깊어지고 있다.
고물가 장기화에 장바구니 가격 부담이 갈수록 커지는 가운데 정월대보름에 챙겨먹는 오곡밥과 부럼 등 재료 가격마저 크게 올랐기 때문이다. 대형마트는 국산 재료 값이 급등하자 일부 품목을 수입산으로 대체하고 있다.
10일 대형마트 업계에 따르면 오곡밥 주재료인 붉은팥, 찹쌀, 서리태, 수수, 차조 등 국산 잡곡 시세가 일제히 상승했다.
특히 잡곡밥에 들어가는 붉은팥 가격이 전년 대비 50%가량 뛰었고 찹쌀도 23% 이상 급등했다. 부럼 재료인 은행과 땅콩 가격 역시 17%가량 올랐다. 국산 건나물도 상황은 마찬가지다.
호박과 고구마순의 가격이 각각 20%, 10% 이상 뛰었고 기획상품으로 내놓는 건나물 4종 세트 역시 전년 대비 평균 5~10% 올랐다.
유통업계에서는 정월대보름 주요 품목 가격이 오른 이유로 재배 면적 축소에 따른 생산량 감소, 폭염 등 이상기후로 인한 작황 부진, 고물가 장기화에 집밥 수요 급증 등의 영향 등을 꼽고 있다.
이에 따라 대형마트들은 고객들의 장바구니 물가 부담을 덜어주기 위해 일부 품목을 수입산으로 대체하고 있다.
붉은팥과 호두, 땅콩 등이 대표적이다.
롯데마트는 오는 12일까지 캐나다·페루산 붉은팥을 1㎏·1.2㎏들이 1봉당 각각 7990원에 선보인다.
또 중국산 볶음 피땅콩(450g)과 미국산 피호두(미국산·300g)를 2개 이상 구매하면 개당 2000원 할인한 각각 5990원에 판다.
홈플러스는 같은 기간 캐나다산 붉은팥을 1봉(600ｇ)당 9990원에 팔고 1봉을 사면 1봉을 덤으로 주는 ‘1+1’ 행사를 연다.
또 미국산 호두(300g)와 중국산 볶음피땅콩(500g)을 4990원에 내놓고, 미국산 피스타치오(100g)를 포함, 대보름 부럼세트(총 280ｇ)를 6990원에 판매한다.
이마트 역시 미국산 피호두(300ｇ)와 중국산 볶음 피땅콩(480ｇ)을 25% 할인한 5235원에 내놓는다.
한편 한국물가정보가 발표한 정월 대보름 주요 10개 품목 가격을 보면 전통시장의 합산 가격은 지난해 대비 6.2% 오른 13만9700원, 대형마트는 8.0% 오른 18만5220원으로 나타났다.
가격 상승 폭이 가장 큰 품목은 오곡밥 재료 중 붉은팥으로 1되(800g)가 전통시장에서는 전년 대비 45.5% 오른 1만6000원, 대형마트에서는 45.0% 오른 2만1920원이었다.
찹쌀은 1되(800g) 가격이 전통시장 기준 3200원으로 지난해보다 23.1% 올랐고, 대형마트에서는 5040원으로 28.6% 뛰었다.
검정콩 1되(720g)는 지난해보다 전통시장·대형마트 판매 가격이 각각 7.1%, 5.2% 올랐다.
부럼 재료 중에서는 은행과 땅콩 가격이 크게 올랐다. 은행 1되(600g) 가격은 전통시장 7000원, 대형마트 9840원으로 지난해보다 각각 16.7%, 15.2% 뛰었다.
땅콩 1되(400g)는 전통시장 가격이 1만원으로 지난해보다 11.1% 올랐고, 대형마트는 1만3560원으로 13.4% 인상됐다.

"""
stream_summary(new_article)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


10일 대형마트 업계에 따르면 고물가 장기화에 장바구니 가격 부담이 갈수록 커지는 가운데 정월대보름에 챙겨먹는 오곡밥과 부럼 등 재료 가격마저 크게 올랐고 일부 품목을 수입산으로 대체하고 있다.

��
